In [1]:
import os
from glob import glob

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Document
from llama_index.llms.ollama import Ollama

# Initialize Ollama LLM
llm = Ollama(model="codestral:latest", request_timeout=10000)

dependecy_folders = [
    'Model',
    "Repository"
]

base_directory = "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice"

source_files = [
    "C:\\GitHub\\microservices-sample\\microservices\\src\microservices\\CartMicroservice\\Controllers\\CartController.cs"
]

test_target_directory = "C:\\GitHub\\microservices-sample\\microservices\\tests\\CartMicroservice.UnitTests"

dependecy_context = ""
for folder in dependecy_folders:
    folder_path = os.path.join(base_directory, folder)
    cs_files = glob(os.path.join(folder_path, "*.cs"))
    for cs_file in cs_files:
        with open(cs_file, "r", encoding="utf-8") as f:
            dependecy_context += f.read().strip() + "\n------\n"
dependecy_context = dependecy_context.strip()

def generate_csharp_unit_tests(service_code: str, test_framework: str = "xUnit"):
    prompt = (
        f"Given the following C# microservice code:\n\n"
        f"{service_code}\n\n"
        f"The dependencies 'Model' and 'Repository' are located in their respective folders: "
        f"'CartMicroservice/Model' and 'CartMicroservice/Repository'.\n"
        f"Here is the context for all dependent classes, separated by '------':\n"
        f"{dependecy_context}\n\n"
        f"Leverage this dependency context while generating comprehensive unit test cases using {test_framework}. "
        f"Include necessary using statements and mock dependencies as needed. "
        f"For all objects, initialize them with concrete sample data, do not leave any placeholders or comments like 'Initialize with test data'. "
        f"Respond ONLY with the code, and do not add any text or explanation before or after the code."
    )
    response = llm.complete(prompt)
    return response

# Example usage:
# service_code = "<your C# microservice code here>"
# print(generate_csharp_unit_tests(service_code))


In [16]:
# Test Single file
for source_path in source_files:
    with open(source_path, "r", encoding="utf-8") as file:
        service_code = file.read()
    unit_tests = generate_csharp_unit_tests(service_code)
    print(f"Unit tests for {source_path}:\n{unit_tests}\n")

Unit tests for C:\GitHub\microservices-sample\microservices\src\microservices\CartMicroservice\Controllers\CartController.cs:
 ```csharp
using CartMicroservice.Controllers;
using CartMicroservice.Model;
using CartMicroservice.Repository;
using Microsoft.AspNetCore.Mvc;
using Moq;
using System;
using System.Collections.Generic;
using Xunit;

namespace CartMicroservice.Tests;

public class CartControllerTests
{
    private readonly Mock<ICartRepository> _mockCartRepo;
    private readonly CartController _cartController;

    public CartControllerTests()
    {
        _mockCartRepo = new Mock<ICartRepository>();
        _cartController = new CartController(_mockCartRepo.Object);
    }

    [Fact]
    public void Get_ReturnsOkResultWithCartItems()
    {
        // Arrange
        var userId = Guid.NewGuid().ToString();
        var cartItems = new List<CartItem>
        {
            new CartItem { CatalogItemId = "1", Name = "Test Item 1", Price = 10, Quantity = 2 },
            // Add mor

In [2]:
import os
from glob import glob
import subprocess
import time


for source_path in source_files:

    with open(source_path, "r", encoding="utf-8") as file:
        service_code = file.read()

    unit_tests = generate_csharp_unit_tests(service_code)

    source_filename = os.path.basename(source_path)
    test_filename = source_filename.replace(".cs", "Tests.cs")
    test_file_path = os.path.join(test_target_directory, test_filename)

    # Remove content before and including ``` from the top, and after and including ``` at the bottom
    test_content = unit_tests.text
    start_idx = test_content.find(" ```csharp")
    if start_idx != -1:
        test_content = test_content[start_idx + 10 :]
    end_idx = test_content.rfind("```")
    if end_idx != -1:
        test_content = test_content[:end_idx]
    test_content = test_content.strip()

    with open(test_file_path, "w", encoding="utf-8") as test_file:
        test_file.write(test_content)

# Try to compile the tests project and update until it compiles
max_attempts = 5
attempt = 0
while attempt < max_attempts:
    result = subprocess.run(
        ["dotnet", "build", test_target_directory],
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("Compilation succeeded.")
        break
    else:
        print(f"Compilation failed (attempt {attempt+1}):\n{result.stderr}")
        # Optionally, you could parse errors and try to regenerate/fix tests here
        # For now, just regenerate all tests and try again
        for source_path in source_files:
            with open(source_path, "r", encoding="utf-8") as file:
                service_code = file.read()
            unit_tests = generate_csharp_unit_tests(service_code)
            source_filename = os.path.basename(source_path)
            test_filename = source_filename.replace(".cs", "Tests.cs")
            test_file_path = os.path.join(test_target_directory, test_filename)
            with open(test_file_path, "w", encoding="utf-8") as test_file:
                test_file.write(unit_tests.text)
        time.sleep(2)
        attempt += 1
else:
    print("Compilation failed after maximum attempts.")

Compilation succeeded.
